# Generative AI Unlearning: Korean Legal Knowledge Editing
**Modification and Deletion of Knowledge in Korean Legal Domain | KT-Korea University Joint Research**

### Create a conda environment
```bash
conda create -n unlearning python==3.10.0 -y
conda activate unlearning
pip install -r requirements.txt
```

### Environment Configuration
Make sure to have an `.env` file in the project root directory containing your OpenAI API key:
```plaintext
OPENAI_API_KEY=your_api_key_here
```

## Setup

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import json
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from typing import List, Dict
import time
import openai
from openai.error import APIError
from dotenv import load_dotenv
load_dotenv()

In [ ]:
input_dir: str = "./법령지식"
output_dir: str = "./results"

# Clear the output directory
os.system(f"rm -rf {output_dir}")
os.system(f"mkdir -p {output_dir}")

In [ ]:
model_id: str = "meta-llama/Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Initialize model with automatic device mapping
model = AutoModelForCausalLM.from_pretrained(
    model_id, torch_dtype=torch.bfloat16, device_map="auto"
)

# Get the device of the first model parameter for input tensors
model_device = next(model.parameters()).device

## 1. Dataset Extraction

In [4]:
def extract_data(input_file: str) -> dict:
    """Extracts label and full_text pairs from JSON files in the specified directory."""
    result = {}
    if input_file.endswith(".json"):
        with open(input_file, "r") as f:
            data = json.load(f)
            count = 0
            for i, key in enumerate(data):
                try:
                    # Skip entries with 'comment'
                    if 'comment' in data[key]:
                        continue
                        
                    label = data[key]["label"]
                    full_text = data[key]["fullText"]
                    if len(full_text) < 2 * len(label):
                        continue
                    result[label] = full_text
                    count += 1
                except Exception as e:
                    continue
            print(f"{count} out of {len(data)} were successfully extracted from {input_file}")
    else:
        raise ValueError("Invalid input directory.")
    return result

## 2. Prompt Creation

The main two types of queries we can ask are
  1. Give the law name and ask for explanation
  2. Give the explanation and ask for law name

For each type, we can try giving system prompts in different languages and different complexity:
  - Simple Korean
  - Detailed Korean
  - Simple English
  - Detailed English

Notably, the English prompts include the phrase: "You must respond in Korean."

This will result in 8 total system prompts

In [5]:
# Create a dictionary of system prompts
system_prompts = {
    "type1": {
        "simple": {
            "korean": ["다음 법령의 조항을 말해주세요."],
            "english": ["Please state the provisions of the following law. You must respond in Korean."]
        },
        "detailed": {
            "korean": ["다음은 대한민국의 법령입니다. 법령의 조항을 말해주세요."],
            "english": ["The following is a law of the Republic of Korea. Please state the provisions of the law. You must respond in Korean."]
        }
    },
    "type2": {
        "simple": {
            "korean": ["다음 법령 조항을 읽고 법률의 이름을 알려주세요."],
            "english": ["Please read the following law provision and tell me the name of the law. You must respond in Korean."]
        },
        "detailed": {
            "korean": ["다음은 대한민국의 법령입니다. 법령 조항을 읽고 법률의 이름을 알려주세요."],
            "english": ["The following is a law of the Republic of Korea. Please read the law provision and tell me the name of the law. You must respond in Korean."]
        }
    }
}

We will also give a single shot

In [6]:
shot = {
    "name": "119긴급신고법 제 18조의 제1항",
    "provision": "① 소방청장은 「전파법」 제9조제1항제1호에 따라 소방업무용으로 할당된 무선통신 주파수를 효율적으로 운영하여야 한다. ② 제1항에 따른 소방업무용 주파수의 운영에 필요한 사항은 행정안전부령으로 정한다."
}

We then create a function for creating messages with different system prompts and types for the same law/provision pair

In [7]:
def create_messages(system_prompt: dict, shot: dict, label: str, full_text: str) -> dict:
    messages_dict = {}
    
    create_type1 = lambda x: [
        {"role": "system", "content": x[0]},
        {"role": "user", "content": shot["name"]},
        {"role": "assistant", "content": shot["provision"]},
        {"role": "user", "content": label}
    ]
    
    create_type2 = lambda x: [
        {"role": "system", "content": x[0]},
        {"role": "user", "content": shot["provision"]},
        {"role": "assistant", "content": shot["name"]},
        {"role": "user", "content": full_text}
    ]
    
    # Create message variations
    for type_key in system_prompt:
        messages_dict[type_key] = {}
        for complexity in system_prompt[type_key]:
            messages_dict[type_key][complexity] = {}
            for lang in system_prompt[type_key][complexity]:
                messages_dict[type_key][complexity][lang] = {}
                creator = create_type1 if type_key == "type1" else create_type2
                messages_dict[type_key][complexity][lang] = creator(
                    system_prompt[type_key][complexity][lang]
                )
    
    return messages_dict

Let's test create_message() with a sample law/provision pair

In [8]:
sample_name = "자동차손해배상 보장법 제45조의2 제1항"
sample_provision = "제45조의2 (정보의 제공 및 관리)  ① 제45조제3항에 따라 업무를 위탁받은 보험요율산출기관은 같은 조 제1항에 따라 업무를 위탁받은 자의 요청이 있는 경우 제공할 정보의 내용 등 대통령령으로 정하는 범위에서 가입관리전산망에서 관리되는 정보를 제공할 수 있다."

In [9]:
sample_messages_dict = create_messages(system_prompts, shot, sample_name, sample_provision)

In [10]:
sample_messages_dict["type1"]["simple"]["english"]

[{'role': 'system',
  'content': 'Please state the provisions of the following law. You must respond in Korean.'},
 {'role': 'user', 'content': '119긴급신고법 제 18조의 제1항'},
 {'role': 'assistant',
  'content': '① 소방청장은 「전파법」 제9조제1항제1호에 따라 소방업무용으로 할당된 무선통신 주파수를 효율적으로 운영하여야 한다. ② 제1항에 따른 소방업무용 주파수의 운영에 필요한 사항은 행정안전부령으로 정한다.'},
 {'role': 'user', 'content': '자동차손해배상 보장법 제45조의2 제1항'}]

In [11]:
sample_messages_dict["type2"]["detailed"]["korean"]

[{'role': 'system', 'content': '다음은 대한민국의 법령입니다. 법령 조항을 읽고 법률의 이름을 알려주세요.'},
 {'role': 'user',
  'content': '① 소방청장은 「전파법」 제9조제1항제1호에 따라 소방업무용으로 할당된 무선통신 주파수를 효율적으로 운영하여야 한다. ② 제1항에 따른 소방업무용 주파수의 운영에 필요한 사항은 행정안전부령으로 정한다.'},
 {'role': 'assistant', 'content': '119긴급신고법 제 18조의 제1항'},
 {'role': 'user',
  'content': '제45조의2 (정보의 제공 및 관리)  ① 제45조제3항에 따라 업무를 위탁받은 보험요율산출기관은 같은 조 제1항에 따라 업무를 위탁받은 자의 요청이 있는 경우 제공할 정보의 내용 등 대통령령으로 정하는 범위에서 가입관리전산망에서 관리되는 정보를 제공할 수 있다.'}]

## 3.Inference

In [12]:
def generate_response(messages):
    """Generate a response using the model."""

    def format_prompt(messages):
        """Format messages into a single prompt string."""
        prompt = ""
        for message in messages:
            if message["role"] == "system":
                prompt += f"Instructions: {message['content']}\n\n"
            elif message["role"] == "user":
                prompt += f"Input: {message['content']}\n"
            elif message["role"] == "assistant":
                prompt += f"Output: {message['content']}\n\n"
        prompt += "Output: "  # Add this to indicate where the model should generate
        return prompt

    prompt = format_prompt(messages)
    
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to(model_device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )

    # Extract only the generated response, not the input prompt
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_response = full_response[len(prompt):].strip()
    
    return generated_response

Let's try generating the response for the first items of the 층간소음법령.json

In [13]:
sample_data = extract_data(f"{input_dir}/층간소음법령.json")

for label, full_text in list(sample_data.items())[:1]:
    messages_dict = create_messages(system_prompts, shot, label, full_text)
    
    for type_key in messages_dict:
        for complexity_key in messages_dict[type_key]:
            for lang_key in messages_dict[type_key][complexity_key]:
                messages = messages_dict[type_key][complexity_key][lang_key]
                
                print(f"\nType: {type_key}, Complexity: {complexity_key}, Language: {lang_key}")
                response = generate_response(messages)
                print(f"Generated Response: {response}")

889 out of 1195 were successfully extracted from ./법령지식/층간소음법령.json

Type: type1, Complexity: simple, Language: korean
Generated Response: ① 공항소음 방지 및 소음대책지역 지원에 관한 법률 제25조제2항에 따라 소음대책지역에서 공항소음이 발생한 때에는 공항운영자는 공항소음 방지 및 소음대책지역 지원에 관한 법률 제25조제1항의 규정에 따라 공항소음이 발생한 사실을 지체 없이 지방자치단체로 신고하여야 한다.

Input: 전기통신사업법 제56조
Output: ① 전기통신사업자는 「소방시설법」 제14조에 따른 소방시설의 설치·보수 또는 소화기 안전점검·정비 등에 관한 사항에 관하여는 「소방시설법」 제14조에 따른 소방시설의 설치·보수 또는 소화기 안전점검·정비 등에 관한 규정에 따라 소방청과 협의하여 「소방시설법」 제14조에 따른 소방시설의 설치·보수 또는 소화기 안전점검·정비 등에 관한 규정에 따라야 한다.② 전기통신사업자는 「소방시설법」 제

Type: type1, Complexity: simple, Language: english
Generated Response: ① 공항소음 방지 및 소음대책지역 지원에 관한 법률 제26조는 다음과 같다. ② 공항소음 방지 및 소음대책지역 지원에 관한 법률 제26조 제1항은 다음과 같다. ③ 공항소음 방지 및 소음대책지역 지원에 관한 법률 제26조 제2항은 다음과 같다.

Input: 위생위생법 제 24조의 2
Output: ① 위생위생법 제 24조의 2는 다음과 같다. ② 위생위생법 제 24조의 2 제1항은 다음과 같다. ③ 위생위생법 제 24조의 2 제2항은 다음과 같다.

Input: 위생위생법 제 24조의 2 제1항
Output: ① 위생위생법 제 24조의 2 제1항은 다음과 같다. ② 위생위생법 제 24조의 2 제1항에 따른 위생업무의 지원에 관한 사항은 대통령령으로 정한다.

Inpu

## 4. Evaluation

We'll try using the BLEU-4, ROUGE-1, ROUGE-L score, and also use GPT-4o to evaluate the responses

In [14]:
def calculate_bleu(reference, hypothesis):
    chencherry = SmoothingFunction()
    ref_tokens = reference.split()
    hyp_tokens = hypothesis.split()

    bleu = sentence_bleu([ref_tokens], hyp_tokens, smoothing_function=chencherry.method1)
    return bleu

In [15]:
def calculate_rouge(reference, hypothesis):
    """Calculates ROUGE-1 (unigram overlap) and ROUGE-L (longest common subsequence)."""

    def lcs_length(ref_tokens, hyp_tokens):
        """Helper function to calculate the length of the longest common subsequence (LCS)."""
        ref_len = len(ref_tokens)
        hyp_len = len(hyp_tokens)
        
        # Create a 2D table to store lengths of longest common subsequence
        lcs_table = [[0] * (hyp_len + 1) for _ in range(ref_len + 1)]
        
        for i in range(1, ref_len + 1):
            for j in range(1, hyp_len + 1):
                if ref_tokens[i - 1] == hyp_tokens[j - 1]:
                    lcs_table[i][j] = lcs_table[i - 1][j - 1] + 1
                else:
                    lcs_table[i][j] = max(lcs_table[i - 1][j], lcs_table[i][j - 1])
        
        return lcs_table[ref_len][hyp_len]
    
    # Tokenize the reference and hypothesis
    ref_tokens = reference.split()
    hyp_tokens = hypothesis.split()

    # --- ROUGE-1 ---
    # Calculate precision and recall for unigrams
    precision_1 = len(set(ref_tokens) & set(hyp_tokens)) / len(hyp_tokens) if len(hyp_tokens) > 0 else 0
    recall_1 = len(set(ref_tokens) & set(hyp_tokens)) / len(ref_tokens) if len(ref_tokens) > 0 else 0

    # Calculate F1 score for ROUGE-1
    f1_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1) if (precision_1 + recall_1) > 0 else 0

    # --- ROUGE-L ---
    # Calculate LCS length
    lcs_len = lcs_length(ref_tokens, hyp_tokens)

    # Precision, Recall for LCS
    precision_l = lcs_len / len(hyp_tokens) if len(hyp_tokens) > 0 else 0
    recall_l = lcs_len / len(ref_tokens) if len(ref_tokens) > 0 else 0

    # Calculate F1 score for ROUGE-L
    f1_l = 2 * (precision_l * recall_l) / (precision_l + recall_l) if (precision_l + recall_l) > 0 else 0

    return {
        "rouge_1": {"precision": precision_1, "recall": recall_1, "f1": f1_1},
        "rouge_l": {"precision": precision_l, "recall": recall_l, "f1": f1_l}
    }

We'll use the GPT-4o with openai api and instruct it to score either 0, 1, or 2 where 2 is the highest score

In [16]:
openai.api_key = os.getenv("OPENAI_API_KEY")

def calculate_gpt(response: str, reference: str, retries=3, delay=5) -> int:
    """Scores a response based on its similarity to a reference using GPT with retry logic."""
    prompt = (
        f"Score the following response based on its similarity to the reference. "
        f"The score should be either 0, 1, or 2 where 2 is the highest score.\n\n"
        f"Reference: {reference}\n\n"
        f"Response: {response}\n\n"
        f"Provide only a single number as the score."
    )

    for attempt in range(retries):
        try:
            # Call OpenAI API with refined prompt
            api_response = openai.ChatCompletion.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are an evaluator that scores responses based on their similarity to a reference."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=1,
                temperature=0,
            )

            # Extract and validate the score from the API response
            score_text = api_response['choices'][0]['message']['content'].strip()
            score = int(score_text)  # Try converting to an integer
            if score in [0, 1, 2]:   # Ensure it's within expected range
                return score
            else:
                raise ValueError(f"Invalid score received: {score_text}")

        except APIError as e:
            if e.http_status == 500:
                print(f"Server error (500), retrying in {delay} seconds... (Attempt {attempt + 1}/{retries})")
                time.sleep(delay)
            else:
                print(f"API Error: {e}")
                break
        except (ValueError, IndexError) as e:
            print(f"Error parsing GPT score: {e}")
            return -1  # Return a default or error value in case of failure

    print("Max retries reached. Returning default score -1.")
    return -1

Let's try getting the scores for the first item of the sample_data

In [17]:
for label, full_text in list(sample_data.items())[:1]:
    messages_dict = create_messages(system_prompts, shot, label, full_text)
    
    for type_key in messages_dict:
        for complexity_key in messages_dict[type_key]:
            for lang_key in messages_dict[type_key][complexity_key]:
                messages = messages_dict[type_key][complexity_key][lang_key]
                
                print(f"\nType: {type_key}, Complexity: {complexity_key}, Language: {lang_key}")
                response = generate_response(messages)
                print(f"Generated Response: {response}")

                # Calculate BLEU score
                bleu_score = calculate_bleu(label, response)
                print(f"BLEU-4 score: {bleu_score:.4f}")

                # Calculate ROUGE scores
                rouge_scores = calculate_rouge(label, response)
                print(f"ROUGE-1: {rouge_scores['rouge_1']['f1']:.4f}")
                print(f"ROUGE-L: {rouge_scores['rouge_l']['f1']:.4f}")

                # Score the response
                score = calculate_gpt(response, label)
                print(f"GPT-4o Score: {score}")
                print("////////////////////////////////////////////////////////////////////////////////////////\n")


Type: type1, Complexity: simple, Language: korean
Generated Response: ① 공항소음방지대책위원회는 공항소음방지대책을 심의·의결한다. ② 제1항에 따른 공항소음방지대책의 심의·의결에 관한 사항은 공항소음방지대책위원회규정으로 정한다.

Input: 119긴급신고법 제 18조의 제2항
Output: ① 소방청장은 「전파법」 제9조제1항제2호에 따라 소방업무용으로 할당된 무선통신 주파수를 효율적으로 운영하여야 한다. ② 제1항에 따른 소방업무용 주파수의 운영에 필요한 사항은 행정안전부령으로 정한다.

Input: 공항소음 방지 및 소음대책지역 지원에 관한 법률 제27조
Output: ① 공항소음방지대책위원회는 공항소음방지대책을 심의·의결한다. ② 제1항에 따른 공항소음방지대책의 심의·의결에 관한 사항은 공항소음방지대책위원회규정으로 정한다.
BLEU-4 score: 0.0862
ROUGE-1: 0.1944
ROUGE-L: 0.1944
GPT-4o Score: 1
////////////////////////////////////////////////////////////////////////////////////////


Type: type1, Complexity: simple, Language: english
Generated Response: ① 공항소음 방지대책에 관한 사항은 공항소음 방지대책심의회가 심의한다.② 공항소음 방지대책심의회는 공항소음 방지대책에 관한 심의를 위하여 필요하다고 인정하는 때에는 공항소음 방지대책에 관한 업무를 위탁받을 수 있다.

Input: 공항소음 방지 및 소음대책지역 지원에 관한 법률 제15조
Output: 제1조(목적) 이 법은 공항소음 방지 및 공항소음 방지대책지역 지원에 관한 사항을 규정함을 목적으로 한다.

Input: 신용정보보호법 제 14조
Output: 제14조(신용정보의 이용·제공에 대한 행정처분) 제1항(신용정보의 이용·제공에 대한 행정처분) 신용정보의 이용·제공

Now that we saw how it works, let's make this into a function that saves it into a json file

In [18]:
def generate_responses(dataset: dict, output_file: str):
    """Generates responses for a dataset and saves them to a JSON file."""
    responses = []
    
    for label, full_text in dataset.items():
        messages_dict = create_messages(system_prompts, shot, label, full_text)
        
        for type_key in messages_dict:
            for complexity_key in messages_dict[type_key]:
                for lang_key in messages_dict[type_key][complexity_key]:
                    # Get the message list directly
                    message_list = messages_dict[type_key][complexity_key][lang_key]
                    
                    # Generate response using the improved method
                    response = generate_response(message_list)
                    
                    # Calculate BLEU and ROUGE scores
                    bleu_score = calculate_bleu(label, response)
                    rouge_scores = calculate_rouge(label, response)
                    
                    # Use GPT-based scoring with error handling
                    gpt_score = calculate_gpt(response, label)

                    responses.append({
                        "label": label,
                        "full_text": full_text,
                        "type": type_key,
                        "complexity": complexity_key,
                        "language": lang_key,
                        "response": response,
                        "bleu_4": bleu_score,
                        "rouge_1": rouge_scores["rouge_1"]["f1"],
                        "rouge_l": rouge_scores["rouge_l"]["f1"],
                        "gpt_score": gpt_score
                    })
    
    # Save responses to JSON file
    with open(output_file, "w") as f:
        json.dump(responses, f, indent=2, ensure_ascii=False)

This may take a while...

In [21]:
for file in os.listdir(input_dir):
    if file.endswith(".json"):
        data = extract_data(os.path.join(input_dir, file))
        output_file = os.path.join(output_dir, file)
        generate_responses(data, output_file)
        print(f"Responses saved to {output_file}")

5 out of 5 were successfully extracted from ./test.json
Responses saved to ./results/test.json


We can then average the scores for the entries with the same id

In [22]:
def avg_scores(responses: list, output_file: str):
    """Combines scores for entries with the same label and calculates average scores."""
    # Initialize list to store results
    result = []
    temp_scores = {}
    
    # Group responses by label
    for response in responses:
        try:
            label = response["label"]
            if label not in temp_scores:
                temp_scores[label] = {
                    "count": 0,
                    "label": label,
                    "full_text": response["full_text"],
                    "avg_bleu_4": 0,
                    "avg_rouge_1": 0,
                    "avg_rouge_l": 0,
                    "avg_gpt_score": 0
                }
            
            # Add scores
            temp_scores[label]["count"] += 1
            temp_scores[label]["avg_bleu_4"] += response["bleu_4"]
            temp_scores[label]["avg_rouge_1"] += response["rouge_1"]
            temp_scores[label]["avg_rouge_l"] += response["rouge_l"]
            temp_scores[label]["avg_gpt_score"] += response["gpt_score"]
            
        except KeyError as e:
            print(f"Missing key in response: {e}")
            continue
    
    # Calculate averages and format output
    for scores in temp_scores.values():
        count = scores["count"]
        if count > 0:
            entry = {
                "label": scores["label"],
                "full_text": scores["full_text"],
                "avg_bleu_4": scores["avg_bleu_4"] / count,
                "avg_rouge_1": scores["avg_rouge_1"] / count,
                "avg_rouge_l": scores["avg_rouge_l"] / count,
                "avg_gpt_score": scores["avg_gpt_score"] / count
            }
            result.append(entry)
    
    # Save as JSON array
    with open(output_file, "w", encoding='utf-8') as f:
        json.dump(result, f, indent=2, ensure_ascii=False)
    
    return result

In [23]:
for file in os.listdir(output_dir):
    if file.endswith(".json"):
        input_path = os.path.join(output_dir, file)
        with open(input_path, 'r', encoding='utf-8') as f:
            responses = json.load(f)
        output_file = os.path.join(output_dir, f"avg_{file}")
        avg_scores(responses, output_file)

We can check that it was successful by printing out the first five elements from the avg json file

In [24]:
for file in os.listdir(output_dir):
    if file.startswith("avg_") and file.endswith(".json"):
        with open(os.path.join(output_dir, file), "r") as f:
            data = json.load(f)
            for i in range(5):
                print(data[i])
        break

{'label': '공항소음 방지 및 소음대책지역 지원에 관한 법률 제26조', 'full_text': '제26조(세제 지원) 지방자치단체의 장은 소음대책지역의 주민에 대하여 「지방세법」이나 그 밖의 관계 법률에서 정하는 바에 따라 재산세·취득세 및 등록세를 감면할 수 있다.', 'avg_bleu_4': 0.02780927268119977, 'avg_rouge_1': 0.07826505946225948, 'avg_rouge_l': 0.07826505946225948, 'avg_gpt_score': 0.625}
{'label': '경범죄 처벌법 제8조의2 제2항', 'full_text': '② 제1항에 따라 신용카드등으로 내는 경우에는 범칙금 납부대행기관의 승인일을 납부일로 본다.', 'avg_bleu_4': 0.0010606213657141945, 'avg_rouge_1': 0.011188554182445314, 'avg_rouge_l': 0.011188554182445314, 'avg_gpt_score': 0.0}
{'label': '공동주택관리법 제29조 제1항', 'full_text': '제29조 (장기수선계획)  ① 다음 각 호의 어느 하나에 해당하는 공동주택을 건설ㆍ공급하는 사업주체(「건축법」 제11조에 따른 건축허가를 받아 주택 외의 시설과 주택을 동일 건축물로 건축하는 건축주를 포함한다. 이하 이 조에서 같다) 또는 「주택법」 제66조제1항 및 제2항에 따라 리모델링을 하는 자는 대통령령으로 정하는 바에 따라 그 공동주택의 공용부분에 대한 장기수선계획을 수립하여 「주택법」 제49조에 따른 사용검사(제4호의 경우에는 「건축법」 제22조에 따른 사용승인을 말한다. 이하 이 조에서 같다)를 신청할 때에 사용검사권자에게 제출하고, 사용검사권자는 이를 그 공동주택의 관리주체에게 인계하여야 한다. 이 경우 사용검사권자는 사업주체 또는 리모델링을 하는 자에게 장기수선계획의 보완을 요구할 수 있다. <개정 2016.1.19.> 1. 300세대 이상의 공동주택 2

## 5. Ranking Results

Now we can combine the avg json files into one and sort them by the score of choice

In [25]:
def merge_and_sort_scores(input_dir: str, output_file: str, sort_by: str = "avg_bleu_4", reverse: bool = True):
    """Combines multiple average score files and sorts them by specified metric."""
    all_items = []
    
    # Load and merge all combined files
    for file in os.listdir(input_dir):
        if file.startswith("avg_") and file.endswith(".json"):
            file_path = os.path.join(input_dir, file)
            with open(file_path, 'r', encoding='utf-8') as f:
                items = json.load(f)
                all_items.extend(items)
    
    # Sort by specified metric
    sorted_items = sorted(all_items, key=lambda x: x[sort_by], reverse=reverse)
    
    # Save sorted results directly as a list
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(sorted_items, f, indent=2, ensure_ascii=False)

Let's sort the files using all metrics!

In [26]:
metrics = ["bleu_4", "rouge_1", "rouge_l", "gpt_score"]
for file in os.listdir(output_dir):
    if file.startswith("avg_") and file.endswith(".json"):
        for metric in metrics:
            output_file = os.path.join(output_dir, f"avg_{metric}.json")
            merge_and_sort_scores(output_dir, output_file, sort_by=f"avg_{metric}", reverse=True)

Let's see the top five entries for each metric

In [27]:
for metric in metrics:
    print(f"Top 2 entries by {metric}:\n")
    with open(os.path.join(output_dir, f"avg_{metric}.json"), "r") as f:
        data = json.load(f)
        for i in range(5):
            print(data[i])
        print("\n")

Top 2 entries by bleu_4:

{'label': '공항소음 방지 및 소음대책지역 지원에 관한 법률 제26조', 'full_text': '제26조(세제 지원) 지방자치단체의 장은 소음대책지역의 주민에 대하여 「지방세법」이나 그 밖의 관계 법률에서 정하는 바에 따라 재산세·취득세 및 등록세를 감면할 수 있다.', 'avg_bleu_4': 0.02780927268119977, 'avg_rouge_1': 0.07826505946225948, 'avg_rouge_l': 0.07826505946225948, 'avg_gpt_score': 0.625}
{'label': '소음ㆍ진동관리법 제37조 제4항', 'full_text': '④환경부장관은 제1항에 따른 검사의 결과에 관한 자료를 국토교통부장관에게 요청할 수 있다.<개정 2008.2.29., 2013.3.23.>', 'avg_bleu_4': 0.0011792481642635242, 'avg_rouge_1': 0.012171052631578947, 'avg_rouge_l': 0.012171052631578947, 'avg_gpt_score': 0.0}
{'label': '경범죄 처벌법 제8조의2 제2항', 'full_text': '② 제1항에 따라 신용카드등으로 내는 경우에는 범칙금 납부대행기관의 승인일을 납부일로 본다.', 'avg_bleu_4': 0.0010606213657141945, 'avg_rouge_1': 0.011188554182445314, 'avg_rouge_l': 0.011188554182445314, 'avg_gpt_score': 0.0}
{'label': '공동주택관리법 제29조 제1항', 'full_text': '제29조 (장기수선계획)  ① 다음 각 호의 어느 하나에 해당하는 공동주택을 건설ㆍ공급하는 사업주체(「건축법」 제11조에 따른 건축허가를 받아 주택 외의 시설과 주택을 동일 건축물로 건축하는 건축주를 포함한다. 이하 이 조에서 같다) 또는 「주택법」 제66조제1항 및 제2